In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_cirrhosis"
cohort = "GSE212047"

# Input paths
in_trait_dir = "../../input/GEO/Liver_cirrhosis"
in_cohort_dir = "../../input/GEO/Liver_cirrhosis/GSE212047"

# Output paths
out_data_file = "../../output/preprocess/Liver_cirrhosis/GSE212047.csv"
out_gene_data_file = "../../output/preprocess/Liver_cirrhosis/gene_data/GSE212047.csv"
out_clinical_data_file = "../../output/preprocess/Liver_cirrhosis/clinical_data/GSE212047.csv"
json_path = "../../output/preprocess/Liver_cirrhosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Opposing Roles of Hepatic Stellate Cell Subpopulations in Hepatocarcinogenesis"
!Series_summary	"Hepatocellular carcinoma (HCC), the fourth leading cause of cancer mortality, develops almost exclusively in patients with chronic liver disease (CLD) and advanced fibrosis. Here we interrogated functions of hepatic stellate cells (HSC), the main source of liver fibroblasts, during hepatocarcinogenPesis. Genetic depletion, activation or inhibition established HSC as tumour-promoting in mouse models of HCC. HSC were enriched in the preneoplastic environment, where they closely interacted with hepatocytes and modulated hepatocarcinogenesis by regulating hepatocyte proliferation and death. Analysis of mouse and human HSC subpopulations and their associated mediators by single cell RNA-sequencing in conjunction with genetic ablation revealed dual functions of HSC in hepatocarcinogenesis. Hepatocyte growth factor, enriched in quiescent and cytokine-producin

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any
import numpy as np

# 1. Gene Expression Data Availability
# Looking at the information, this is a dataset about hepatic stellate cells (HSC)
# The background mentions RNAseq and microarray data, so gene expression data is likely available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Liver_cirrhosis)
# Looking at the characteristics, there's no explicit mention of cirrhosis
# However, the background mentions "fibrotic livers" and "cirrhotic" samples
# There's no clear indication in the characteristics that distinguishes cirrhotic vs non-cirrhotic samples
trait_row = None  # No direct trait data available

# For age
# There's no age information in the sample characteristics
age_row = None

# For gender
# There's no gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# Since we don't have trait, age, or gender data, we'll define placeholder functions

def convert_trait(value):
    return None

def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save Metadata
# Initial filtering - we have gene data but no trait data
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We skip this step since trait_row is None (no clinical data available)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Liver_cirrhosis/GSE212047/GSE212047-GPL6246_series_matrix.txt.gz
Gene data shape: (35556, 8)
First 20 gene/probe identifiers:
Index(['10338001', '10338002', '10338003', '10338004', '10338005', '10338006',
       '10338007', '10338008', '10338009', '10338010', '10338011', '10338012',
       '10338013', '10338014', '10338015', '10338016', '10338017', '10338018',
       '10338019', '10338020'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be probe IDs from an Illumina BeadChip array (GPL6246)
# They are numeric identifiers rather than human gene symbols
# These will need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
{'ID': ['10344614', '10344616', '10344618', '10344620', '10344622'], 'GB_LIST': ['AK145513,AK145782', nan, nan, 'AK140060', nan], 'SPOT_ID': ['chr1:3054233-3054733', 'chr1:3102016-3102125', 'chr1:3276323-3277348', 'chr1:3680571-3680912', 'chr1:4771131-4772199'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000067.6', 'NC_000067.6', 'NC_000067.6', 'NC_000067.6', 'NC_000067.6'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['3054233', '3102016', '3276323', '3680571', '4771131'], 'RANGE_STOP': ['3054733', '3102125', '3277348', '3680912', '4772199'], 'total_probes': [33.0, 25.0, 25.0, 25.0, 8.0], 'gene_assignment': ['ENSMUST00000160944 // Gm16088 // predicted gene 16088 // --- // --- /// ENSMUST00000120800 // Gm14300 // predicted gene 1430


Checking for columns containing transcript or gene related terms:
Column 'seqname' may contain gene-related information
Sample values: ['chr1', 'chr1', 'chr1']
Column 'gene_assignment' may contain gene-related information
Sample values: ['ENSMUST00000160944 // Gm16088 // predicted gene 16088 // --- // --- /// ENSMUST00000120800 // Gm14300 // predicted gene 14300 // --- // --- /// ENSMUST00000179907 // G430049J08Rik // RIKEN cDNA G430049J08 gene // --- // --- /// AK145513 // Gm2889 // predicted gene 2889 // 18 A1|18 // 100040658', 'ENSMUST00000082908 // Gm26206 // predicted gene, 26206 // --- // ---', '---']


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for mapping
# From the annotation data, we can see:
# - 'ID' contains the probe identifiers that match the gene expression data
# - 'gene_assignment' contains gene information (in a complex format)

# 2. Get a gene mapping dataframe
print("Creating gene mapping dataframe...")
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("Sample of mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
print("\nApplying gene mapping to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("Sample of mapped gene expression data:")
print(gene_data.head())

# Check the first few genes in the mapped data
print("\nFirst few genes after mapping:")
print(gene_data.index[:10])


Creating gene mapping dataframe...
Mapping dataframe shape: (35556, 2)
Sample of mapping data:
         ID                                               Gene
0  10344614  ENSMUST00000160944 // Gm16088 // predicted gen...
1  10344616  ENSMUST00000082908 // Gm26206 // predicted gen...
2  10344618                                                ---
3  10344620  AK140060 // Gm10568 // predicted gene 10568 //...
4  10344622                                                ---

Applying gene mapping to gene expression data...


Gene expression data shape after mapping: (59765, 8)
Sample of mapped gene expression data:
      GSM6508430  GSM6508431  GSM6508432  GSM6508433  GSM6508434  GSM6508435  \
Gene                                                                           
A-     32.055253   32.089315   32.205307   32.230057   32.252179   32.511957   
A-52   21.294962   21.030563   21.167686   20.719513   20.579016   20.896422   
A-B     9.060165    9.180475    9.150914    9.143384    9.115320    9.151546   
A-B1    5.907612    5.843892    5.785713    5.791814    5.800345    5.805113   
A-C     7.939185    7.697075    7.891581    7.727363    7.914639    7.847250   

      GSM6508436  GSM6508437  
Gene                          
A-     32.177730   32.289920  
A-52   20.387463   20.340958  
A-B     9.172020    9.237754  
A-B1    5.855846    5.817073  
A-C     8.102693    7.904450  

First few genes after mapping:
Index(['A-', 'A-52', 'A-B', 'A-B1', 'A-C', 'A-D', 'A-I', 'A-II', 'A-IV',
       'A-V'],
      dtyp

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Check if trait_row is None (indicating no clinical data is available)
if trait_row is None:
    print("No clinical data available for this dataset, skipping clinical data processing.")
    
    # Validate and save cohort information with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use this data without clinical features
        df=pd.DataFrame(),  # Empty DataFrame since we have no linked data
        note="Dataset contains gene expression data from cell lines with HCV infection, which is not appropriate for liver cirrhosis trait analysis."
    )
    
    print("Dataset is not usable for liver cirrhosis analysis due to lack of clinical data. No linked data file saved.")
else:
    # If clinical data is available, proceed with the linking and processing
    # 2. Link the clinical and genetic data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty after missing value handling")

    # 5. Validate and save cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note="Dataset contains gene expression data for liver fibrosis progression, which is relevant to liver cirrhosis research."
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")

Gene data shape before normalization: (59765, 8)
Gene data shape after normalization: (1043, 8)
Normalized gene expression data saved to ../../output/preprocess/Liver_cirrhosis/gene_data/GSE212047.csv
No clinical data available for this dataset, skipping clinical data processing.
Abnormality detected in the cohort: GSE212047. Preprocessing failed.
Dataset is not usable for liver cirrhosis analysis due to lack of clinical data. No linked data file saved.
